In [1]:
import requests
import json
import pandas as pd


headers = {
    "Accept": "*/*",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}


def get_response(url):
    s = requests.Session()
    response = s.get(url=url, headers=headers)
    
    return response


# читання та запис *.json файлу
def get_json(url):
    with open("result_page.json", "w", encoding="utf-8") as file:
        json.dump(get_response(url).json(), file, indent=4, ensure_ascii=False)

        
# отримання кількості сторінок та елементів відбору 
def get_collection(url):
    data = get_response(url).json()
    last_page = data.get("last_page")
    total_items = data.get("total_items")
    
    return last_page, total_items


''' 
В процесі взаємодії є кілька варіантів запитів: 
 - через меню підвалу (відбір іде по вибраній категорії, при цьому локація фіксована - "Київ, Україна")
 - через меню навігаціїї (відбір іде по вибраній категорії, при цьому локація визначаеться по геолокації користувача або та, яка раніше була вказана, а також за замовчуванням відбір тільки по салонам)
 - через блок пошуку (відбір іде по вибраній категорії, при цьому локація визначаеться по геолокації користувача або та, яка раніше була вказана)
В результаті отримуємо дані в форматі *.json, але різні набори та адреси з відповідями. 
В прикладі використав відповід, отриману через блок пошуку.  
'''
url = "https://app.mybeauty.ua/api/v1/salon/nearest?perPage=10&page=1&latitude=50.4501&longitude=30.5234&searchText=%D0%91%D0%B0%D1%80%D0%B1%D0%B5%D1%80%D1%88%D0%BE%D0%BF%D0%B8&address=%D0%9A%D0%B8%D1%97%D0%B2,%20Ukraine"
# Список з кількостю відібраних сторінок та елементів
pages = get_collection(url)
# Перша сторінка відбору
page = 1
# Кількість елементів на сторінці
item_end = 10
# Пустий словник для збереження результату відбору
item_description = {'name': [],
                    'nameService': [],
                   'siteUrl': []}

# в циклі проходимо по всім сторінкам відбору (Барбершопи в Києві)
for i in range(0, pages[0]):
    url_page = (f"https://app.mybeauty.ua/api/v1/salon/nearest?perPage=10&page={page + i}&latitude=50.4501&longitude=30.5234&searchText=%D0%91%D0%B0%D1%80%D0%B1%D0%B5%D1%80%D1%88%D0%BE%D0%BF%D0%B8&address=%D0%9A%D0%B8%D1%97%D0%B2,%20Ukraine")
    data = get_response(url_page).json()
    
#   визначаємо кількість елементів на останній сторінці відбору
    if i + 1 == pages[0]:
        item_end = pages[1] % item_end

#   в циклі проходимо по всім елементам відбору на сторінці
    for item in range(0, item_end):
        item_description['name'].append(data[str(item)]['name'])
        item_description['siteUrl'].append(data[str(item)]['translations']['original']['siteUrl'])
        
#       в циклі проходимо по всім послугам елемента відбору та вносимо їх в список
        lst_service = []
        for j in range(0, int(len(data[str(item)]['masterServiceDirections']))):
            lst_service.append(data[str(item)]['masterServiceDirections'][j]['translations']['original']['name'])
        item_description['nameService'].append(lst_service)

# друк результату відбору
print(item_description)




{'name': ['ES Beauty Bar', 'New Look (вул. Прорізна, 13)', 'Барбершоп The Head', 'Приватний кабінет Артема Іванюка', 'Салон краси "Beauty Pro"', 'PIED-DE-POULE (ул. Б. Хмельницкого, 19/21)', 'Творчий простір Ar4istail', 'PIED-DE-POULE (вул. Ярославів вал, 19)', 'Барбершоп Rebel', 'Салон краси Vie Privee', 'PIED-DE-POULE (вул. Шота Руставелі, 20)', 'Барбершоп FIRM (вул. Антоновича, 3)', 'Студія краси Bella Vita', 'Барбершоп Rival', 'Простір Black Rabbit', 'Барбершоп ДАБОРОДА', 'Барбершоп Barboss', 'Jolly beauty & care', 'Чеховський', 'Студія краси Wit', 'Kika-style (вул. Антоновича, 18)', 'Салон краси Spa Diva', 'BIDA HAIR PROJECT', 'Барбершоп Ruffade (Кловський узвіз, 9/2)', 'Екосалон краси Maximum', 'Барбершоп TheSketch', 'Zhe. Unisex Beauty Bar', 'ILe De Beaute', 'Барбершоп Dust', 'Kika-Style (вул. Князів Острозьких (Московська), 29А)', 'M15 Барбершоп', 'Diplomat Barbershop', 'Студія Volkhovskaya', 'Студія B28', 'Leo Barberry', 'Салон краси Hair Guide', 'Приватний кабінет Сугірій Нат

In [3]:
# pip install pandas

In [2]:
pd.DataFrame(item_description)

,name,nameService,siteUrl
0,ES Beauty Bar,"[Барбершопы, Брови, Косметология, Макияж, Мани...",None
1,"New Look (вул. Прорізна, 13)","[Барбершопы, Брови, Макияж, Маникюр, Парикмахе...",None
2,Барбершоп The Head,[Барбершопы],None
3,Приватний кабінет Артема Іванюка,[Барбершопы],None
4,"Салон краси ""Beauty Pro""","[Барбершопы, Брови, Макияж, Маникюр, Парикмахе...",None
...,...,...,...
73,"Barberos (пр-т. Петра Григоренко, 28)",[Барбершопы],None
74,"Барбершоп Heisenberg (вул. Градинська, 9)","[Барбершопы, Удаление волос]",None
75,Барбершоп Birmingham,[Барбершопы],None
76,Барбершоп Stone House,"[Барбершопы, Парикмахер]",None
